In [1]:
import matplotlib.pyplot as plt
from pyomo.environ import *
from pyomo.dae import *

import numpy as np
from scipy.interpolate import interp2d
import pandas as pd 
from itertools import permutations, product, combinations

from fim_doe import *
from pyomo.contrib.sensitivity_toolbox.sens import sipopt
from idaes.apps.uncertainty_propagation.sens import get_dsdp

    'pyomo.common.sorting.sorted_robust'.  Please update your import.
    (deprecated in 6.1) (called from <frozen importlib._bootstrap>:228)


## Define fixed bed model 

In [2]:
from fixed_bed_model_indexed_dynamic import *

alpha option:
1 / (1 + exp(-x))
    Import ComponentSet from pyomo.common.collections.  (deprecated in 5.7.1)
    (called from <frozen importlib._bootstrap>:228)


In [3]:
def add_model(doe, timepoints, time_start=0):
    '''
    add variables, equations and discretize the model time
    Argument:
        timepoints: the timesteps
        time_start: where the timesteps start. For self defined timepoints it's 0. For experimental data it's -10.
    '''
    add_variables(doe, timesteps=timepoints, start=time_start)
    
    add_equations(doe)
    
    print ('the number of timepoints is', NFEt)
    
def discretize(doe, no_points=68, collo=False):
    '''
    no_points: how many time invertals to divide. For self defined timepoints it's 69. For experimental data it's 68. 
    here it's 68
    '''
    if collo:
        TransformationFactory('dae.collocation').apply_to(doe, nfe=no_points, ncp=3, scheme='LAGRANGE-RADAU', wrt=doe.t)
    else:
        TransformationFactory('dae.finite_difference').apply_to(doe, nfe=no_points, scheme='BACKWARD', wrt=doe.t)
    
    
def initialize(doe, init_point):
    '''
    Argument: 
        init_point: initial point, csv file
    '''
    # Initialize 
    print('The init point is', init_point)    
    store = pd.read_csv(init_point)
    position_max = store['position'].max()
    store['position'] = store['position'] / position_max

    initial_bed_csv(doe, store)
    fix_initial_bed(doe)
    
def generate_exp(feed, bath, y):
    dv_dict_overall = {'temp_feed': {0: feed}, 'temp_bath': {0:bath}, 'yfeed': {0: y}}
    return dv_dict_overall

In [4]:
NFEt = 68
time_scale = 3200
collo = False
exp_time_ = time_points(NFEt, time_scale)

scena_all = {'scena-name': [0], 'fitted_transport_coefficient':212, 'ua': np.log(5.0E6)}

initialize_point = '/Users/wangjialu/dowlinglab/sorption-fixed-bed/20200729_energy.csv'
#init_point = '20210916_feed313_bath293.csv'
#init_point = '/Users/wangjialu/dowlinglab/sorption-fixed-bed/fixed_bed_initial/T353y20_5e3.csv'
#init_point = '20210916_feed293_bath313_5e4.csv'
#init_point = 'init_test.csv'
#init_point = 'T353y20_5e3.csv'

In [5]:
def model_integrate(scena, args_list, exp_time=exp_time_, init_point=initialize_point):
    test = create_model(scena, temp_feed=293.15, temp_bath=303.15, y=0.2, doe_model=args_list[0], k_aug=args_list[1], opt=False, diff=0)
    add_model(test, exp_time)
    discretize(test)
    initialize(test, init_point)
    return test


In [6]:
# choose from model and k_aug 
model_opt = 'doe'

if (model_opt == 'doe'):
    args_ = [True, False]
    
elif (model_opt =='kaug'):
    args_ = [True, True]
    

parameter_dict = {'fitted_transport_coefficient':212, 'ua': np.log(5.0E6)}

#dv_pass = {'temp_feed': [0], 'temp_bath':[0], 'yfeed': [0]}
dv_pass = {'temp_feed': None, 'temp_bath':None , 'yfeed': None}

measurement = ['FCO2']
t_measure = exp_time_


In [7]:
# empty prior
prior_all = np.zeros((2,2))

prior_pass=np.asarray(prior_all)

#L_initials = np.linalg.cholesky(prior_pass)
#print(L_initials)

print('The prior information FIM:', prior_pass)
print('Prior Det:', np.linalg.det(prior_pass))
print('Eigenvalue of the prior experiments FIM:', np.linalg.eigvals(prior_pass))
print('Eigenvalue of the prior experiments FIM:', np.linalg.eigh(prior_pass)[1])

The prior information FIM: [[0. 0.]
 [0. 0.]]
Prior Det: 0.0
Eigenvalue of the prior experiments FIM: [0. 0.]
Eigenvalue of the prior experiments FIM: [[1. 0.]
 [0. 1.]]


In [8]:
# choose from 'simultaneous', 'sequential', 'sipopt'
#sensi_opt = 'simultaneous_finite'
sensi_opt = 'sequential_finite'
##sensi_opt = 'sequential_sipopt'
#sensi_opt = 'sequential_kaug'
#sensi_opt = 'direct_kaug'

# feed, bath, y
exp1 = generate_exp(313.15, 313.15, 0.15)

In [ ]:
doe_object = DesignOfExperiments(parameter_dict, dv_pass,
                                 measurement, t_measure, model_integrate,
                                prior_FIM=prior_pass, discretize_model=None, args=args_, measurement_extra_index = [19])

if_s = True 

result = doe_object.compute_FIM(exp1, mode=sensi_opt, FIM_store_name = 'fixed_bed.csv',
                               scale_nominal_param_value = if_s, formula='central')

if sensi_opt == 'simultaneous_finite':
    result.extract_FIM(doe_object.m, doe_object.design_timeset,
                                                          doe_object.square_result, doe_object.obj_opt)
else: 
    result.calculate_FIM(doe_object.jac, doe_object.design_values)

Sensitivity information is scaled by its corresponding parameter nominal value.
This scenario: {'fitted_transport_coefficient': {0: 214.12}, 'ua': {0: 15.424948470398375}, 'jac-index': {'fitted_transport_coefficient': [0], 'ua': [0]}, 'eps-abs': {'fitted_transport_coefficient': 2.1412, 'ua': 0.15424948470398375}, 'scena-name': [0]}
The inlet feed density is 41.0275814575221 [mol/m3]
the number of timepoints is 68
The init point is /Users/wangjialu/dowlinglab/sorption-fixed-bed/20200729_energy.csv
Model # of time grid is 69 , initial point # of time grid is 69
Parameters:  214.12 , 15.424948470398375
temp_feed is fixed at  313.15
temp_bath is fixed at  313.15
yfeed is fixed at  0.15
Ipopt 3.13.2: linear_solver=ma57
halt_on_ampl_error=yes


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For m

 187r 0.0000000e+00 7.99e-02 4.70e+02  -3.3 4.22e-01    -  5.28e-02 2.94e-02f  1
 188r 0.0000000e+00 7.73e-02 4.39e+02  -3.3 4.48e-01    -  2.64e-02 4.39e-02f  1
 189r 0.0000000e+00 7.54e-02 4.34e+02  -3.3 3.48e-01    -  5.27e-02 2.75e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 0.0000000e+00 7.37e-02 4.20e+02  -3.3 7.89e-01  -0.2 1.70e-02 2.46e-02f  1
 191r 0.0000000e+00 4.66e-02 2.57e+02  -3.3 6.14e-02   1.1 7.38e-02 3.81e-01f  1
 192r 0.0000000e+00 4.59e-02 4.94e+02  -3.3 7.92e-01    -  5.78e-05 1.72e-02f  1
 193r 0.0000000e+00 3.72e-02 4.04e+02  -3.3 1.59e-01   0.6 2.12e-01 2.27e-01f  1
 194r 0.0000000e+00 3.69e-02 3.99e+02  -3.3 3.99e+00   0.1 1.22e-02 1.11e-02f  1
 195r 0.0000000e+00 3.62e-02 3.90e+02  -3.3 8.29e-01    -  1.07e-02 2.30e-02f  1
 196r 0.0000000e+00 3.59e-02 3.86e+02  -3.3 6.96e-01   0.6 2.66e-02 1.03e-02f  1
 197r 0.0000000e+00 2.95e-02 5.59e+02  -3.3 8.88e-02   1.0 2.63e-02 1.94e-01f  1
 198r 0.0000000e+00 2.91e-02

Reallocating memory for MA57: lfact (2559697)
   1  0.0000000e+00 1.25e+02 2.56e+02  -1.0 1.25e+02    -  7.48e-01 2.98e-03h  1
   2  0.0000000e+00 1.25e+02 2.46e+06  -1.0 1.25e+02    -  2.89e-01 2.98e-05h  1
   3r 0.0000000e+00 1.25e+02 1.00e+03   2.1 0.00e+00    -  0.00e+00 1.49e-07R  2
   4r 0.0000000e+00 1.23e+02 3.59e+04   2.1 1.22e+02    -  1.31e-02 1.70e-02f  1
   5r 0.0000000e+00 5.70e+04 3.49e+06   2.1 1.22e+02    -  5.23e-02 6.10e-02f  1
   6r 0.0000000e+00 3.59e+04 4.00e+06   2.1 3.44e-01   6.0 3.50e-01 4.39e-01f  1
   7r 0.0000000e+00 3.54e+04 3.94e+06   2.1 4.60e-01   6.4 3.96e-01 1.34e-02h  1
   8r 0.0000000e+00 3.09e+04 3.06e+06   2.1 4.63e-01   5.9 1.00e+00 1.36e-01h  1
   9r 0.0000000e+00 2.19e+04 4.14e+06   2.1 9.26e-01   5.5 2.34e-01 3.29e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10r 0.0000000e+00 1.90e+04 3.43e+06   2.1 4.85e-01   5.9 1.74e-01 1.43e-01h  1
  11r 0.0000000e+00 1.79e+04 4.01e+06   2.1 4.16e-01   7.2 1.22

 185r 0.0000000e+00 3.13e-02 2.07e+02  -2.1 1.60e-01   1.4 2.36e-01 3.45e-01f  1
 186r 0.0000000e+00 3.10e-02 2.04e+02  -2.1 1.29e+00   0.9 2.65e-02 9.96e-03f  1
 187r 0.0000000e+00 2.63e-02 2.92e+02  -2.1 1.15e-01   1.3 4.47e-01 1.51e-01f  1
 188r 0.0000000e+00 1.27e-02 2.44e+02  -2.1 3.27e-02   1.7 9.34e-01 5.22e-01f  1
 189r 0.0000000e+00 1.78e-02 2.48e+02  -2.1 1.33e+00    -  5.58e-02 3.13e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 0.0000000e+00 1.83e-02 2.66e+02  -2.1 1.07e+00    -  2.85e-02 1.73e-02f  1
 191r 0.0000000e+00 1.87e-02 2.58e+02  -2.1 6.99e+00    -  3.66e-03 5.23e-03f  1
 192r 0.0000000e+00 2.58e-02 5.34e+02  -2.1 7.62e-01    -  1.97e-01 6.84e-02f  1
 193r 0.0000000e+00 3.42e-02 4.53e+02  -2.1 4.14e-01    -  1.51e-01 1.52e-01f  1
 194r 0.0000000e+00 3.40e-02 4.46e+02  -2.1 1.99e+00    -  1.53e-02 1.24e-02f  1
 195r 0.0000000e+00 3.39e-02 4.42e+02  -2.1 2.54e+00    -  1.03e-02 1.22e-02f  1
 196r 0.0000000e+00 3.91e-02

Reallocating memory for MA57: lfact (2559598)
   1  0.0000000e+00 1.07e+02 2.56e+02  -1.0 1.07e+02    -  7.48e-01 2.98e-03h  1
   2  0.0000000e+00 1.07e+02 2.46e+06  -1.0 1.07e+02    -  2.89e-01 2.98e-05h  1
   3r 0.0000000e+00 1.07e+02 1.00e+03   2.0 0.00e+00    -  0.00e+00 1.49e-07R  2
   4r 0.0000000e+00 1.05e+02 2.78e+04   2.0 1.32e+02    -  1.52e-02 1.90e-02f  1
   5r 0.0000000e+00 9.66e+03 5.85e+05   2.0 1.31e+02    -  1.32e-01 5.64e-02f  1
   6r 0.0000000e+00 6.37e+03 3.30e+06   2.0 3.30e-01   6.0 3.23e-01 3.97e-01f  1
   7r 0.0000000e+00 6.30e+03 3.25e+06   2.0 4.25e-01   6.4 2.72e-01 1.22e-02h  1
   8r 0.0000000e+00 5.20e+03 2.96e+06   2.0 4.28e-01   5.9 1.00e+00 1.87e-01f  1
   9r 0.0000000e+00 4.03e+03 3.13e+06   2.0 7.71e-01   5.5 1.99e-01 2.48e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10r 0.0000000e+00 2.76e+03 6.33e+06   2.0 4.45e-01   5.9 3.80e-01 3.64e-01h  1
  11r 0.0000000e+00 2.66e+03 6.16e+06   2.0 4.91e-01   6.3 2.09

 187r 0.0000000e+00 1.68e-02 1.67e+02  -2.2 8.19e-02   0.7 3.49e-01 5.29e-01f  1
 188r 0.0000000e+00 1.65e-02 1.66e+02  -2.2 4.67e-01   0.2 1.18e-01 8.20e-02f  1
 189r 0.0000000e+00 1.88e-02 1.65e+02  -2.2 6.15e-01    -  6.71e-02 5.14e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 0.0000000e+00 1.30e-02 6.99e+01  -2.2 8.25e-02   0.7 7.12e-01 6.39e-01f  1
 191r 0.0000000e+00 1.54e-02 1.13e+02  -2.2 3.47e-01    -  1.19e-01 1.01e-01f  1
 192r 0.0000000e+00 2.12e-02 9.06e+01  -2.2 1.99e-01    -  2.59e-01 2.89e-01f  1
 193r 0.0000000e+00 1.76e-02 1.33e+02  -2.2 8.60e-02    -  1.00e+00 6.29e-01f  1
 194r 0.0000000e+00 1.04e-03 4.42e-01  -2.2 1.46e-02    -  1.00e+00 1.00e+00f  1
 195r 0.0000000e+00 5.43e-05 2.33e-02  -2.2 3.18e-04    -  1.00e+00 1.00e+00h  1
 196r 0.0000000e+00 3.22e-03 2.84e+02  -3.3 1.10e-01    -  1.00e+00 3.01e-01f  1
 197r 0.0000000e+00 2.96e-03 2.55e+02  -3.3 4.00e-02    -  1.00e+00 5.57e-01f  1
 198r 0.0000000e+00 1.84e-04

In [ ]:
print(doe_object.dsdp)

In [ ]:
print('======Result summary======')
print('Four design criteria log10() value:')
print('A-optimality:', np.log10(result.trace))
print('D-optimality:', np.log10(result.det))
print('E-optimality:', np.log10(result.min_eig))
print('Modified E-optimality:', np.log10(result.cond))

In [ ]:
doe_object.models[0]

In [ ]:
breakthrough_modify2(doe_object.models[2])

In [ ]:
def breakthrough_modify2(m, file=None, source="lab"):
    '''
    Draw breakthrough curve of the Pyomo model
    
    Arguments:
        m: Pyomo model
        file: when source = 'computer', this is where the computer experimental data is stored. Otherwise, its default is None
        source: if computer, it is plotting the computer experiment data. If lab, its comparing the experiments from the lab
    
    Return: None 
    
    Other: plot
    
    '''
    
    
    # unit: mol/m3
    outlet_den, _, _, _ = extract3d(m, m.C, 'CO2')
    
    # unit: cm/s
    outlet_vel, _, _, _ = extract2d(m, m.v)
    
    outlet_n2, _, _, _ = extract3d(m, m.C, 'N2')
    
    model_temp, _,_,_ = extract2d(m,m.temp)
    
    
    T = []
    for j in m.t:
        T.append(value(j))
        t_final = T[-1]
        
    
    if source == "lab":
        break_wvu = pd.read_csv('/Users/wangjialu/dowlinglab/sorption-fixed-bed/breakthrough_wvu.csv')
    
        exp = pd.read_csv('/Users/wangjialu/dowlinglab/sorption-fixed-bed/co2_breakthrough.csv')
        
        data_c1 = exp['FCO2']
        data_c = np.asarray(data_c1)
        data_t1 = exp['time']  
        data_t = np.asarray(data_t1*60+10)
        new_data = np.interp(T, data_t, data_c)
    
        fco2 = []
        for t in m.t:
            fco2.append(value(m.FCO2[0,19,t]))
        
        yco2 = []
        for i in range(len(fco2)):
            yco2.append(fco2[i]/fco2[-1])
            
        # real value of FCO2. for verification
        comp = outlet_den[-1,:]*outlet_vel[-1,:]/(outlet_den[-1, -1]*outlet_vel[-1,-1])
    
        plt.plot(exp['time']*60, exp['FCO2'],'b.', color='r', label='Experimental data')
        #plt.plot(T, outlet_den[-1,:]*outlet_vel[-1,:]/(outlet_den[-1, -1]*outlet_vel[-1,-1]), label='Model prediction')
        plt.plot(T, yco2, label='Model prediction')
        
        plt.xlabel('time [s]')
        plt.ylabel('Normalized outlet gas density of CO\N{SUBSCRIPT TWO} ')
        plt.title('Breakthrough curve')
        #plt.savefig('break_tr%.fua%.f.png'%(tr,ua))
        plt.legend()
        plt.show()
        
    elif source == "computer": 
        
        sol = pd.read_csv(file)
    
        #for i in range(len(time_exp)):    
        #    yco2_exp[time_exp[i]] = sol['FCO2'][i]
        #    temp_mid_exp[time_exp[i]] = sol['temp_mid'][i]
        #    temp_end_exp[time_exp[i]] = sol['temp_end'][i]
        
        exp_fco2 = np.asarray(sol['FCO2'].values.tolist())
        exp_temp_mid = np.asarray(sol['temp_mid'].values.tolist())
        exp_temp_end = np.asarray(sol['temp_end'].values.tolist())
        
        plt.plot(T, exp_fco2/exp_fco2[-1], label = 'Experimental data')
        plt.plot(T, outlet_den[-1,:]*outlet_vel[-1,:]/(outlet_den[-1, -1]*outlet_vel[-1,-1]), label='Model prediction')
        plt.xlabel('time [s]')
        plt.ylabel('Normalized outlet gas density of CO\N{SUBSCRIPT TWO} ')
        plt.title('Breakthrough curve')
        #plt.savefig('break_tr%.fua%.f.png'%(tr,ua))
        plt.legend()
        plt.show()
        
        
        plt.plot(T, exp_temp_mid, label='Experimental data of middle T')
        plt.plot(T, model_temp[10,:], label='Model prediction of middle T')
        plt.plot(T, exp_temp_end, label='Experimental data of end T')
        plt.plot(T, model_temp[19,:], label='Model prediction of end T')
        plt.xlabel('time [s]')
        plt.ylabel('Temperature [K]')
        plt.title('Temperature model prediction and experimental data')
        #plt.savefig('break_tr%.fua%.f.png'%(tr,ua))
        plt.legend()
        plt.show()